In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib

df = pd.read_csv('./titanic.csv')
df.head(10)
df.isnull().sum()

#To visulaize the data
import seaborn as sns
sns.barplot(x='Embarked',y='Survived',hue='Sex',data=df)

sns.pointplot(x="Pclass",y="Survived",hue="Sex",data=df, markers=['o','+'],linestyles=['-',"--"])

def simplify_ages(df):
    bins = (-1,0,5,12,18,25,35,60)
    group_names = ['Unknown','Baby','Child','Teenager','Young Adult','Adult','Senior']
    categories = pd.cut(df.Age,bins,labels=group_names)
    df.Age = categories.astype(str)
    return df

simplify_ages(df3)
sns.barplot(x="Age",y="Survived",hue="Sex",data=df3)

def simplify_fare(df):
    bins = (0,8,14,31,512)
    group_names=['1_quartile','2_quartile','3_quartile','High Fare']
    categories = pd.cut(df.Fare,bins,labels=group_names)
    df.Fare = categories.astype(str)
    return df
simplify_fare(df3)    
sns.barplot(x="Fare",y="Survived",hue="Sex",data=df3)

#One Hot-Encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df3['Fare'] = le.fit_transform(df3['Fare'])
df3['Sex'] = le.fit_transform(df3['Sex'])
df3['Age'] = le.fit_transform(df3['Age'])
df3

#to remove NA's
df.isnull().sum()
df['Embarked'].unique()
df1 =  df[~df['Embarked'].isnull()]
df1.shape
df1.isnull().sum()     

def fill_nas_in_age(df):
    df_without_nas = pd.DataFrame()
    df_with_nas = pd.DataFrame()
    check  = pd.DataFrame()
    
    df_with_nas_class3 = df[df['Pclass']==3][df['Age'].isnull()]
    df_with_nas_class2 = df[df['Pclass']==2][df['Age'].isnull()]
    df_with_nas_class1 = df[df['Pclass']==1][df['Age'].isnull()]
    df_without_nas = df[~df['Age'].isnull()]
    df_with_nas_class3['Age'] = 25
    df_with_nas_class2['Age'] = 30
    df_with_nas_class1['Age'] = 38
    
    df_without_nas = pd.concat([df_with_nas_class3,df_without_nas])
    df_without_nas = pd.concat([df_with_nas_class2,df_without_nas])
    df_without_nas = pd.concat([df_with_nas_class1,df_without_nas])
    
    return df_without_nas


df2 = fill_nas_in_age(df1)
df2.isnull().sum()

#To remove useless columns to build model
df4 = df3.drop(['Cabin','Ticket','SibSp','PassengerId','Name','Embarked','Parch','female','male'],axis = 'columns')
df4.head()

#to segregate dependent variables(y) and independent variables(x)
x = df4.drop('Survived',axis = 'columns')
y = df4.Survived

#To split data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state = 40)

#test 5 classification algorithm from sklearn and get the best scores of each algorithm
rom sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
def find_best_model_using_gridsearch(x,y):
    algos = {
        'logistic_regression':{
            'model':LogisticRegression(),
            'params':{
                'C':np.logspace(-3,3,7),
                'penalty':["l2"],
                'max_iter':[100],
                'fit_intercept':[True,False]
            }
        },
        'naive_bayes_Multi':{
            'model':MultinomialNB(),
            'params':{
                'fit_prior':[True,False],
                'alpha':(1,0.1,0.01)
                
            }
        },
        'Random_forest':{
            'model':RandomForestClassifier(),
            'params':{
            'n_estimators':[4,6,9],
            'max_features':['log2','sqrt','auto'],
            'criterion':['entropy','gini'],
            'max_depth':[2,3,5,10,12,15],
            'min_samples_split':[2,3,5],
            'min_samples_leaf':[1,5,8]
            }
        },
        'Decision_Tree':{
            'model':tree.DecisionTreeClassifier(),
            'params':{
                'criterion':['entropy','gini'],
                'splitter':['best','random'],
                'max_depth':[2,3,5,8,12,16],
                'min_samples_split':[2,3,5],
                'min_samples_leaf':[1,5,8]
            }
        },
        'naive_bayes_Gaussian':{
            'model':GaussianNB(),
            'params':{
                
                'var_smoothing':[1e-09]
                
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits = 5,test_size=0.2,random_state=0)
    for algo_name,config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(x,y)
        scores.append({
            'model':algo_name,
            'best_score':gs.best_score_,
            'best_params': gs.best_params_
    })
    
    return pd.DataFrame(scores,columns=['model','best_score',"best_params"])
    
#to get the best algorithm 
find_best_model_using_gridsearch(x,y)

